In [ ]:
import os

os.chdir("/content/drive/MyDrive/DREAM")
!ls

 Data				       finetune.ipynb	   SECOND
 dictionary_73_11.pt		       FIRST		   submit.ipynb
 dictionary.pt			       Forms		  'Submit results.gdoc'
 dream				       FOURTH		   THIRD
'Dream Olfactory Mixtures Plan.gdoc'   ft_examine.ipynb    Trials
 eval.ipynb			       FT_FIRST		   vectorizing.ipynb
'Eval trials.gsheet'		       hparams.ipynb
 examine.ipynb			       new_dictionary.pt


In [ ]:
!rm -rf dream
!git clone https://github.com/laurahsisson/dream.git
import sys

sys.path.append("dream")

Cloning into 'dream'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 199 (delta 120), reused 131 (delta 55), pack-reused 0
Receiving objects: 100% (199/199), 358.81 KiB | 8.75 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
!pip install torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
!pip install rdkit
!pip install ogb
!pip install optuna
!pip install torchmetrics

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html


In [ ]:
import random

random.seed(42)

In [ ]:
import json

with open("Data/odor_pair_266.json") as f:
    pair_dataset = json.load(f)

pair_dataset = [d for d in pair_dataset if d["mol1"] and d["mol2"]]

len(pair_dataset), pair_dataset[0]

(266758,
 {'mol1': 'CC1=CC2C(C2(C)C)CC1C(=O)C',
  'mol2': 'CCCC(CC)O',
  'blend_notes': ['herbal']})

In [ ]:
import dream.pairdata

# Building the dataset from raw GS data is faster than loading the premade dataset from Drive.
full_dataset = dream.pairdata.make(pair_dataset)
len(full_dataset), full_dataset[0]

[16:31:51] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[16:31:52] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[16:31:52] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
[16:31:52] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[16:31:52] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'
100%|██████████| 266758/266758 [03:45<00:00, 1183.66it/s]


(266450,
 {'pair': ('CC1=CC2C(C2(C)C)CC1C(=O)C', 'CCCC(CC)O'),
  'graph': BlendData(x=[20, 9], edge_index=[2, 40], edge_attr=[40, 3], mol_batch=[20], blend_batch=[2]),
  'notes': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.])})

In [ ]:
import torch
import dream.tokenizer

dictionary = torch.load("dictionary_73_11.pt")
tokenizer = dream.tokenizer.GraphTokenizer(dictionary)

In [ ]:
import copy
from tqdm.notebook import tqdm


def tokenize_dataset_entry(ds):
    ds = copy.copy(ds)
    ds["graph"] = tokenizer.tokenize(ds["graph"])
    return ds


full_dataset_tokenized = [
    tokenize_dataset_entry(ds) for ds in tqdm(full_dataset, smoothing=0)
]
full_dataset[0]["graph"], full_dataset_tokenized[0]["graph"]

  0%|          | 0/266450 [00:00<?, ?it/s]

(BlendData(x=[20, 9], edge_index=[2, 40], edge_attr=[40, 3], mol_batch=[20], blend_batch=[2]),
 BlendData(x=[20], edge_index=[2, 40], edge_attr=[40], mol_batch=[20], blend_batch=[2]))

In [ ]:
import sklearn
import sklearn.model_selection

test_size = 0.2
train_idx, test_idx = sklearn.model_selection.train_test_split(
    list(range(len(full_dataset))), test_size=test_size, random_state=42
)
train_dataset = [full_dataset[i] for i in train_idx]
test_dataset = [full_dataset[i] for i in test_idx]
train_dataset_tokenized = [full_dataset_tokenized[i] for i in train_idx]
test_dataset_tokenized = [full_dataset_tokenized[i] for i in test_idx]
len(train_dataset), len(test_dataset), len(train_dataset_tokenized), len(
    test_dataset_tokenized
)

(213160, 53290, 213160, 53290)

In [ ]:
def make_trial_id():
    TRIAL_ID_LENGTH = 6
    trial_id = str(uuid.uuid4())[:TRIAL_ID_LENGTH]
    return trial_id


hparams_id = "6ddd10"
# hparams_id = make_hparams_id()

In [ ]:
hyperparameter_trial_name = "fifth_hparam_trials"
folder_name = os.path.join("Trials", hyperparameter_trial_name)
os.makedirs(folder_name, exist_ok=True)


torch.save(dictionary, os.path.join(folder_name, "dictionary.pt"))
# Step 2: Specify the path to the SQLite database file
db_path = os.path.join(folder_name, f"{hparams_id}.db")
storage_name = f"sqlite:///{db_path}"
storage_name

'sqlite:///Trials/fifth_hparam_trials/6ddd10.db'

In [ ]:
import dream.mpnn
import dream.encoder
import uuid
import json
import time
import torch_geometric as tg
import numpy as np
import torchmetrics
import dream.olfactor
import gc

MAX_MINUTES = 120
MAX_TIME = MAX_MINUTES * 60
MAX_SIZE = 5e5
MIN_SIZE = 1e5

# I KNOW THIS WHOLE THING IS A HUGE MESS.


def get_config(trial):
    # 2**x
    min_dim = 3
    max_dim = 7

    num_mpnns = trial.suggest_int("num_mpnns", 1, 3)
    mpnn_configs = []

    for i in range(num_mpnns):
        nof = int(2 ** trial.suggest_int(f"node_out_feats{i}", min_dim, max_dim))
        ehf = int(2 ** trial.suggest_int(f"edge_hidden_feats{i}", min_dim, max_dim))
        nsmp = trial.suggest_int(f"num_step_message_passing{i}", 1, 5)
        cfg = dream.mpnn.Config(
            node_out_feats=nof, edge_hidden_feats=ehf, num_step_message_passing=nsmp
        )
        mpnn_configs.append(cfg)

    do_tokenize = trial.suggest_categorical(
        "do_tokenize", [True, False]
    )  # Best is usually True
    if do_tokenize:
        graph_tokenizer = tokenizer
    else:
        graph_tokenizer = None

    # Although for pre-training, single-stage models perform the best, for the
    # larger cardinality of blends, twos-stage models work better.
    return {
        "epochs": 150,  # Maybe more?
        "train_size": len(train_idx),
        "test_size": len(test_idx),
        # Model definition
        "num_mpnns": num_mpnns,
        "graph_tokenizer": graph_tokenizer,
        "do_tokenize": do_tokenize,
        "do_edge_update": trial.suggest_categorical("do_edge_update", [True, False]),
        "do_two_stage": trial.suggest_categorical("do_two_stage", [True, False]),
        "embedding_dim_x": int(
            2 ** trial.suggest_int("embedding_dim_x", min_dim, max_dim)
        ),
        "embedding_dim_edge_attr": int(
            2 ** trial.suggest_int("embedding_dim_edge_attr", min_dim, max_dim)
        ),
        "dropout": trial.suggest_float("dropout", 0, 0.5),
        "num_sabs": trial.suggest_int("num_sabs", 1, 12),
        "mpnn_configs": mpnn_configs,
        "heads": int(2 ** trial.suggest_int("heads", 0, 4)),
        "aggr_mode": trial.suggest_categorical("aggr_mode", ["mean", "max"]),
        "act_mode": "relu",
        # Pretraining procedure
        "pretrain_lr": trial.suggest_float("pretrain_lr", 5e-4, 1e-2, log=True),
        "warmup": 0.05,  # Not retuning for every epochs. This works well enough.
        "weight_decay": 0.05,  # This works really well, consistently.
        # Fixed parameters:
        "bsz": 2**11,  # A100
        "betas": (0.9, 0.999),
        "trial_id": make_trial_id(),
    }


def make_datasets(do_tokenize):
    if do_tokenize:
        return train_dataset_tokenized, test_dataset_tokenized
    else:
        return train_dataset, test_dataset


def make_scheduler(optimizer, warmup, total_steps):
    warmup_steps = int(warmup * total_steps)
    cooldown_steps = total_steps - warmup_steps
    # Define the warmup and cooldown schedulers
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=1e-10, end_factor=1.0, total_iters=warmup_steps
    )
    cooldown_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=1.0, end_factor=1e-10, total_iters=cooldown_steps
    )
    # Chain the schedulers together with SequentialLR
    return torch.optim.lr_scheduler.SequentialLR(
        optimizer,
        schedulers=[warmup_scheduler, cooldown_scheduler],
        milestones=[warmup_steps],
    )


def do_pretrain(trial, config):
    st = time.perf_counter()
    encoder = dream.encoder.Encoder(**config)
    olfactor = dream.olfactor.Olfactor(encoder, dream.pairdata.NOTES_DIM)
    encoder_size = olfactor.encoder.count_parameters()["total"]
    assert encoder_size <= MAX_SIZE
    assert encoder_size >= MIN_SIZE

    olfactor.cuda()
    print(
        f"Trial ID = {config['trial_id']}", encoder.count_parameters(), config, sep="\n"
    )

    train, test = make_datasets(config["do_tokenize"])
    train_loader = tg.loader.DataLoader(train, batch_size=config["bsz"], shuffle=True)
    # Though we could evaluate with a larger bsz, we don't strictly need to.
    test_loader = tg.loader.DataLoader(test, batch_size=config["bsz"], shuffle=True)

    optimizer = torch.optim.AdamW(
        encoder.parameters(),
        lr=config["pretrain_lr"],
        weight_decay=config["weight_decay"],
        betas=config["betas"],
    )

    total_steps = config["epochs"] * len(train_loader)
    scheduler = make_scheduler(optimizer, config["warmup"], total_steps)

    scaler = torch.cuda.amp.GradScaler()

    def make_ckpt_data():
        data = config | {
            "train_loss": train_losses,
            "test_loss": test_losses,
            "test_aurocs": test_aurocs,
            "lr": lrs,
        }
        data["count_parameters"] = encoder.count_parameters()
        del data["graph_tokenizer"]
        return data

    def save(path):
        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path, "config.json"), "w") as f:
            json.dump(make_ckpt_data(), f)
        torch.save(olfactor.state_dict(), os.path.join(path, "olfactor.pt"))

    def predict(batch):
        batch = {k: v.cuda() for k, v in batch.items() if k != "pair"}
        olfact = olfactor(batch["graph"])
        return olfact["logits"], batch["notes"]

    def calc_loss(batch):
        logits, targets = predict(batch)
        return torch.nn.functional.binary_cross_entropy_with_logits(logits, targets)

    def calc_test_metrics(loader):
        tls = []
        auroc_metric = torchmetrics.classification.MultilabelAUROC(
            dream.pairdata.NOTES_DIM, average="micro"
        )
        with torch.no_grad():
            for tbatch in loader:
                with torch.cuda.amp.autocast():
                    logits, targets = predict(tbatch)

                    auroc_metric.update(logits, targets.int())

                    btl = torch.nn.functional.binary_cross_entropy_with_logits(
                        logits, targets
                    )
                    tls.append(btl.cpu().detach().numpy())

        auroc = auroc_metric.compute().cpu().detach().numpy()
        return {"auroc": auroc, "loss": np.mean(tls)}

    save_fraction = 0.25
    test_losses = []
    test_aurocs = []
    train_losses = []
    lrs = []
    btls = []

    best_loss = 0
    patience = 0

    step = 0
    with tqdm(total=total_steps, smoothing=0) as pbar:
        for epoch in range(config["epochs"]):
            olfactor.train()
            btls = []
            for batch in train_loader:
                with torch.cuda.amp.autocast():
                    loss = calc_loss(batch)

                btls.append(loss.cpu().detach())

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                optimizer.zero_grad()
                scheduler.step()
                lrs.append(optimizer.param_groups[0]["lr"])

                step += 1
                pbar.update(1)
                if (step + 1) % max(int(total_steps * save_fraction), 1) == 0:
                    path = os.path.join(
                        "Trials",
                        hyperparameter_trial_name,
                        config["trial_id"],
                        f"step_{step}",
                    )
                    save(path)

                if time.perf_counter() - st > MAX_TIME:
                    print(
                        f"Stopping early after {time.perf_counter() - st:.0f} seconds."
                    )
                    break

            train_losses.append(np.mean(btls).item())

            olfactor.eval()
            test_metrics = calc_test_metrics(test_loader)
            test_losses.append(test_metrics["loss"].item())
            test_aurocs.append(test_metrics["auroc"].item())

            trial.report(test_metrics["auroc"].item(), epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

    path = os.path.join("Trials", hyperparameter_trial_name, config["trial_id"])
    save(path)

    return olfactor, test_aurocs[-1]


def train_model(trial, display_model=False):
    gc.collect()
    torch.cuda.empty_cache()

    start = time.perf_counter()
    config = get_config(trial)
    olfactor, final_auroc = do_pretrain(trial, config)

    path = os.path.join("Trials", hyperparameter_trial_name, config["trial_id"])
    with open(os.path.join(path, "config.json"), "r") as f:
        data = json.load(f)

    with open(os.path.join(path, "config.json"), "w") as f:
        data["auroc"] = final_auroc
        data["time"] = time.perf_counter() - start
        json.dump(data, f)

    return final_auroc


####
import optuna


num_trials = 128

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.QMCSampler(
        qmc_type="halton", scramble=True, warn_independent_sampling=False
    ),
    pruner=optuna.pruners.HyperbandPruner(),
    storage=storage_name,
    load_if_exists=True,
)
study.optimize(train_model, n_trials=num_trials, catch=(AssertionError,))

print("DONE")
print("CONFIG:", study.best_trial.params)
print("METRICS:", study.best_trial.values)

<ipython-input-20-a7cbc546752d>:228: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  sampler=optuna.samplers.QMCSampler(qmc_type="halton",scramble=True,warn_independent_sampling=False),
[W 2024-07-30 16:51:21,313] No seed is provided for `QMCSampler` and the seed is set randomly. If you are running multiple `QMCSampler`s in parallel and/or distributed  environment, the same seed must be used in all samplers to ensure that resulting samples are taken from the same QMC sequence. 
[I 2024-07-30 16:51:21,366] A new study created in RDB with name: no-name-e73c1bbb-478f-4468-8a91-25aa4fdda36e


Trial ID = bee41a
{'total': 155424, 'embed': 208, 'convs': [46688, 3184, 46176], 'readout': 59168}
{'epochs': 150, 'train_size': 213160, 'test_size': 53290, 'num_mpnns': 3, 'graph_tokenizer': None, 'do_tokenize': False, 'do_edge_update': True, 'do_two_stage': False, 'embedding_dim_x': 8, 'embedding_dim_edge_attr': 32, 'dropout': 0.43616139074687266, 'num_sabs': 5, 'mpnn_configs': [Config(node_out_feats=32, edge_hidden_feats=32, num_step_message_passing=2), Config(node_out_feats=8, edge_hidden_feats=16, num_step_message_passing=5), Config(node_out_feats=32, edge_hidden_feats=32, num_step_message_passing=1)], 'heads': 1, 'aggr_mode': 'mean', 'act_mode': 'relu', 'pretrain_lr': 0.0021594072786617624, 'warmup': 0.05, 'weight_decay': 0.05, 'bsz': 2048, 'betas': (0.9, 0.999), 'trial_id': 'bee41a'}


  0%|          | 0/15750 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
[I 2024-07-30 17:59:30,536] Trial 0 finished with value: 0.5397272109985352 and parameters: {'num_mpnns': 3, 'node_out_feats0': 5, 'edge_hidden_feats0': 5, 'num_step_message_passing0': 2, 'node_out_feats1': 3, 'edge_hidden_feats1': 4, 'num_step_message_passing1': 5, 'node_out_feats2': 5, 'edge_hidden_feats2': 5, 'num_step_message_passing2': 1, 'do_tokenize': False, 'do_edge_update': True, 'do_two_stage': False, 'embedding_dim_x': 3, 'embedding_dim

Trial ID = 376720
{'total': 413024, 'embed': 144, 'convs': [87744, 3696, 171616], 'readout': 149824}
{'epochs': 150, 'train_size': 213160, 'test_size': 53290, 'num_mpnns': 3, 'graph_tokenizer': None, 'do_tokenize': False, 'do_edge_update': True, 'do_two_stage': False, 'embedding_dim_x': 8, 'embedding_dim_edge_attr': 16, 'dropout': 0.4176649695002777, 'num_sabs': 3, 'mpnn_configs': [Config(node_out_feats=32, edge_hidden_feats=64, num_step_message_passing=2), Config(node_out_feats=8, edge_hidden_feats=16, num_step_message_passing=2), Config(node_out_feats=64, edge_hidden_feats=32, num_step_message_passing=3)], 'heads': 8, 'aggr_mode': 'max', 'act_mode': 'relu', 'pretrain_lr': 0.0007270744232219083, 'warmup': 0.05, 'weight_decay': 0.05, 'bsz': 2048, 'betas': (0.9, 0.999), 'trial_id': '376720'}


...

Trial ID = 861b4d
{'total': 359304, 'embed': 5504, 'convs': [303680, 5216, 17416], 'readout': 27488}
{'epochs': 150, 'train_size': 213160, 'test_size': 53290, 'num_mpnns': 3, 'graph_tokenizer': <dream.tokenizer.GraphTokenizer object at 0x7c76ebde7dc0>, 'do_tokenize': True, 'do_edge_update': False, 'do_two_stage': False, 'embedding_dim_x': 64, 'embedding_dim_edge_attr': 64, 'dropout': 0.1606750825282372, 'num_sabs': 2, 'mpnn_configs': [Config(node_out_feats=64, edge_hidden_feats=64, num_step_message_passing=2), Config(node_out_feats=8, edge_hidden_feats=32, num_step_message_passing=4), Config(node_out_feats=32, edge_hidden_feats=8, num_step_message_passing=3)], 'heads': 4, 'aggr_mode': 'mean', 'act_mode': 'relu', 'pretrain_lr': 0.0023449513997060123, 'warmup': 0.05, 'weight_decay': 0.05, 'bsz': 2048, 'betas': (0.9, 0.999), 'trial_id': '861b4d'}


  0%|          | 0/15750 [00:00<?, ?it/s]

[I 2024-07-31 05:54:45,311] Trial 112 pruned. 


Trial ID = 7b5174
{'total': 142712, 'embed': 704, 'convs': [19480], 'readout': 122528}
{'epochs': 150, 'train_size': 213160, 'test_size': 53290, 'num_mpnns': 1, 'graph_tokenizer': None, 'do_tokenize': False, 'do_edge_update': True, 'do_two_stage': False, 'embedding_dim_x': 64, 'embedding_dim_edge_attr': 16, 'dropout': 0.45335800935750553, 'num_sabs': 11, 'mpnn_configs': [Config(node_out_feats=32, edge_hidden_feats=8, num_step_message_passing=4)], 'heads': 16, 'aggr_mode': 'max', 'act_mode': 'relu', 'pretrain_lr': 0.00462068832432136, 'warmup': 0.05, 'weight_decay': 0.05, 'bsz': 2048, 'betas': (0.9, 0.999), 'trial_id': '7b5174'}


  0%|          | 0/15750 [00:00<?, ?it/s]

In [ ]:
from google.colab import runtime

runtime.unassign()